### This file creates 'AgentShare' parameters, which is used in 'Technodata.csv'

In [27]:
# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# pip install pdfplumber
import pandas as pd
from pathlib import Path
# import numpy as np


In [29]:
# the key is the consumption/demand name is MUSE
# the value is the name used in the HouseholdElectricitySurveyFinalReportissue4.pdf
non_heat_enduses_mapping = {
    'RES.COOKING': 'Cooking',
    'RES.COOLING': 'Cold appliances',
    'RES.CONSUMER-ELECTRONICS.TV': 'Audiovisual',
    'RES.LIGHTING': 'Lighting',
    'RES.REFRIGERATORS': 'Cold appliances',
    'RES.FREEZERS': 'Cold appliances',
    'RES.COMPUTERS': 'ICT',
    'RES.WET.APPLIANCES': 'Washing/Drying',
    'RES.OTHER': 'Other'
}

heating_enduses_mapping = {
    'RES.HOT-WATER': 'Water heating',
    'RES.SPACE-HEAT': 'Heating'
}
non_heat_enduses = list(non_heat_enduses_mapping.keys())
heating_enduses = list(heating_enduses_mapping.keys())

In [30]:
#### Ofgem data folder
data_folder = Path.cwd().parent / "Ofgem_Archetype"

# read the tables in the excel file
df_ofgem = pd.read_excel(data_folder / "Ofgem energy consumer archetypes2024_Tables1.xlsx", sheet_name="original").dropna()

# keep only the columns we need
col_to_keep = ['Archetype','Average Annual Elec consumption (kWh)','Average Annual Gas consumption (kWh)', 'Main heating Fuel']

df_ofgem = df_ofgem[col_to_keep]

# display the data
df_ofgem.head()

,Archetype,Average Annual Elec consumption (kWh),Average Annual Gas consumption (kWh),Main heating Fuel
0,A1,2742,10933,Mains gas
1,A2,2849,9464,Mains gas
2,A3,3519,10622,Mains gas
3,B4,4811,0,Electricity
4,B5,6597,0,Electricity/Other (Solid fuel/LPG)


In [31]:
technodata_df = pd.read_csv( Path.cwd().parent /"Buildings/MUSE_Files" / "Technodata.csv")

# Step 1: Identify columns with "new" in the "Unit" row, a.k.a the Agent columns
columns_to_drop = technodata_df.loc[0][technodata_df.loc[0] == "new"].index

# Step 2: Drop these old Agent columns from the DataFrame
technodata_df.drop(columns=columns_to_drop, inplace=True)

# Step 3: Extract the "Unit" row, which will be added back to the DataFrame later
unit_row = technodata_df.iloc[[0]]

# Step 4: Remove the "Unit" row from technodata_df for merging
# technodata_df_no_unit = technodata_df.iloc[1:].reset_index(drop=True)

# Step 5: split the df into two dfs based on the "EndUse" column
# Define the condition for splitting
non_heat_use = technodata_df['EndUse'].str.contains('|'.join(non_heat_enduses), case=False, na=False)
heating_use = technodata_df['EndUse'].str.contains('|'.join(heating_enduses), case=False, na=False)

# Split the DataFrame
df_non_heat = technodata_df[non_heat_use]
df_heating = technodata_df[heating_use]


(1) Non-heat

In [32]:
from create_non_heat_share import non_heat_capacity_share

df1 = non_heat_capacity_share(df_non_heat, df_ofgem,non_heat_enduses_mapping)


In [33]:
df1

,key_0,ProcessName,RegionName,Time,cap_par,cap_exp,fix_par,fix_exp,var_par,var_exp,...,F15,F16,G17,G18,H19,H20,I21,I22,J23,J24
1,RES.COOKING,RCEOTHER00 [RES.COOKING: .00.OTHER.KETTLE.MICR...,UK,2010,61.177609885735,1,0.0,1,0,1,...,0.061982,0.023923,0.053139,0.047673,0.044188,0.033383,0.043229,0.049751,0.048136,0.067746
2,RES.COOKING,RCEOTHER01 [RES.COOKING: .01.OTHER.KETTLE.MICR...,UK,2010,61.177609885735,1,0.0,1,0,1,...,0.061982,0.023923,0.053139,0.047673,0.044188,0.033383,0.043229,0.049751,0.048136,0.067746
3,RES.COOKING,RCHEHOB00 [RES.COOKING: .00.HOB.ELECTRIC.],UK,2010,164.642857142857,1,8.23214285714286,1,0,1,...,0.061982,0.023923,0.053139,0.047673,0.044188,0.033383,0.043229,0.049751,0.048136,0.067746
4,RES.COOKING,RCHEHOB01 [RES.COOKING: .01.HOB.ELECTRIC.],UK,2010,164.642857142857,1,8.23214285714286,1,0,1,...,0.061982,0.023923,0.053139,0.047673,0.044188,0.033383,0.043229,0.049751,0.048136,0.067746
5,RES.COOKING,RCHEOVEN00 [RES.COOKING: .00.OVEN.ELECTRIC.],UK,2010,164.642857142857,1,8.23214285714286,1,0,1,...,0.061982,0.023923,0.053139,0.047673,0.044188,0.033383,0.043229,0.049751,0.048136,0.067746
6,RES.COOKING,RCHEOVEN01 [RES.COOKING: .01.OVEN.ELECTRIC.],UK,2010,164.642857142857,1,8.23214285714286,1,0,1,...,0.061982,0.023923,0.053139,0.047673,0.044188,0.033383,0.043229,0.049751,0.048136,0.067746
7,RES.COOKING,RCHGHOB00 [RES.COOKING: .00.HOB.GAS.],UK,2010,175.714285714286,1,8.78571428571429,1,0,1,...,0.061982,0.023923,0.053139,0.047673,0.044188,0.033383,0.043229,0.049751,0.048136,0.067746
8,RES.COOKING,RCHGHOB01 [RES.COOKING: .01.HOB.GAS.],UK,2010,175.714285714286,1,8.78571428571429,1,0,1,...,0.061982,0.023923,0.053139,0.047673,0.044188,0.033383,0.043229,0.049751,0.048136,0.067746
9,RES.COOKING,RCHGOVEN00 [RES.COOKING: .00.OVEN.GAS.],UK,2010,175.714285714286,1,8.78571428571429,1,0,1,...,0.061982,0.023923,0.053139,0.047673,0.044188,0.033383,0.043229,0.049751,0.048136,0.067746
10,RES.COOKING,RCHGOVEN01 [RES.COOKING: .01.OVEN.GAS.],UK,2010,175.714285714286,1,8.78571428571429,1,0,1,...,0.061982,0.023923,0.053139,0.047673,0.044188,0.033383,0.043229,0.049751,0.048136,0.067746


(2) heating enduse

In [34]:
from create_heating_share import heating_capacity_share
df2 = heating_capacity_share(df_heating, df_ofgem)


c:\Users\jyang8\MUSE_models\MUSE_buildings\python_script\create_heating_share.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  technodata.loc[:, 'MappedFuelType'] = technodata['Fuel'].apply(map_fuel_to_type)
c:\Users\jyang8\MUSE_models\MUSE_buildings\python_script\create_heating_share.py:94: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.loc[rows, columns] = data.loc[rows, columns].fillna(0)


In [35]:
df2

,ProcessName,RegionName,Time,cap_par,cap_exp,fix_par,fix_exp,var_par,var_exp,MaxCapacityAddition,...,J23,G17,H19,J24,B5,G18,B4,C8,F15,F16
13,RCHPEA-CCG00 [RES.CHP.EXISTING-AVERAGE: .00.CO...,UK,2010,423.684210526316,1,16.8421052631579,1,0,1,2.0,...,0.081446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,RCHPEA-CCG01 [RES.CHP.EXISTING-AVERAGE: .01.CO...,UK,2010,423.684210526316,1,16.8421052631579,1,0,1,100000000.0,...,0.081446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,RCHPEA-CCH01 [RES.CHP.EXISTING-AVERAGE: .01.CO...,UK,2010,423.684210526316,1,16.8421052631579,1,0,1,100000000.0,...,0.0,0.0,0.0,0.0,0.233978,0.0,0.170633,0.198156,0.244121,0.153112
16,RCHPEA-FCH01 [RES.CHP.EXISTING-AVERAGE: .01.FU...,UK,2010,6456.04931207143,1,531.651428571429,1,0,1,100000000.0,...,0.0,0.0,0.0,0.0,0.233978,0.0,0.170633,0.198156,0.244121,0.153112
17,RCHPEA-STW01 [RES.CHP.EXISTING-AVERAGE: .01.ST...,UK,2010,534.534534534534,1,24.024024024024,1,0,1,100000000.0,...,0.0,0.0,0.0,0.0,0.233978,0.0,0.170633,0.198156,0.244121,0.153112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,RWEAWHTRG00 [RES.WATER.EXISTING-AVERAGE: .00.N...,UK,2010,6.10855195224853,1,0.610855195224853,1,0,1,2.0,...,0.081446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
132,RWEAWHTRG01 [RES.WATER.EXISTING-AVERAGE: .01.N...,UK,2010,6.10855195224853,1,0.610855195224853,1,0,1,100000000.0,...,0.081446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133,RWNASA01 [RES.WATER.NEW-AVERAGE: .01.STANDALON...,UK,2010,0.000593142810946307,1,0.000593142810946307,1,0,1,100000000.0,...,0.0,0.0,0.0,0.0,0.233978,0.0,0.170633,0.198156,0.244121,0.153112
134,RWNAWHTRE01 [RES.WATER.NEW-AVERAGE: .01.ELC.RE...,UK,2010,2.09714772485452,1,0.209714772485452,1,0,1,100000000.0,...,0.0,0.0,0.0,0.0,0.233978,0.0,0.170633,0.198156,0.244121,0.153112


(3) combine dfs together

In [36]:
set(df1.columns) 

{'A1',
 'A2',
 'A3',
 'B4',
 'B5',
 'B6',
 'C7',
 'C8',
 'C9',
 'D10',
 'D11',
 'D12',
 'E13',
 'E14',
 'EndUse',
 'F15',
 'F16',
 'Fuel',
 'G17',
 'G18',
 'H19',
 'H20',
 'I21',
 'I22',
 'InterestRate',
 'J23',
 'J24',
 'MaxCapacityAddition',
 'MaxCapacityGrowth',
 'ProcessName',
 'RegionName',
 'ScalingSize',
 'TechnicalLife',
 'Time',
 'TotalCapacityLimit',
 'UtilizationFactor',
 'cap_exp',
 'cap_par',
 'efficiency',
 'fix_exp',
 'fix_par',
 'key_0',
 'var_exp',
 'var_par'}

In [37]:
set(unit_row.columns)

{'EndUse',
 'Fuel',
 'InterestRate',
 'MaxCapacityAddition',
 'MaxCapacityGrowth',
 'ProcessName',
 'RegionName',
 'ScalingSize',
 'TechnicalLife',
 'Time',
 'TotalCapacityLimit',
 'UtilizationFactor',
 'cap_exp',
 'cap_par',
 'efficiency',
 'fix_exp',
 'fix_par',
 'var_exp',
 'var_par'}

In [38]:
# Step 1: Identify newly added columns from enduse_shares_df
new_columns = set(df1.columns) - set(unit_row.columns) 
new_columns

{'A1',
 'A2',
 'A3',
 'B4',
 'B5',
 'B6',
 'C7',
 'C8',
 'C9',
 'D10',
 'D11',
 'D12',
 'E13',
 'E14',
 'F15',
 'F16',
 'G17',
 'G18',
 'H19',
 'H20',
 'I21',
 'I22',
 'J23',
 'J24',
 'key_0'}

In [39]:
# Step 1: Identify newly added columns from enduse_shares_df
new_columns = new_columns = set(df1.columns) - set(unit_row.columns) 

# Step 2: Update the Unit row
unit_row = unit_row.copy()
for col in new_columns:
    unit_row[col] = 'new'

# Step 3: Add the updated Unit row back to the top
final_merged_df = pd.concat([unit_row, df1,df2], ignore_index=True)



In [40]:
final_merged_df

,ProcessName,RegionName,Time,cap_par,cap_exp,fix_par,fix_exp,var_par,var_exp,MaxCapacityAddition,...,I22,J24,B6,D12,key_0,A3,G18,J23,F16,H19
0,Unit,-,year,MGBP2020/PJ_a,-,MGBP2020/PJ_a,-,MGBP2020/PJ,-,PJ,...,new,new,new,new,new,new,new,new,new,new
1,RCEOTHER00 [RES.COOKING: .00.OTHER.KETTLE.MICR...,UK,2010,61.177609885735,1,0.0,1,0,1,2.0,...,0.049751,0.067746,0.032162,0.041976,RES.COOKING,0.037377,0.047673,0.048136,0.023923,0.044188
2,RCEOTHER01 [RES.COOKING: .01.OTHER.KETTLE.MICR...,UK,2010,61.177609885735,1,0.0,1,0,1,100000000.0,...,0.049751,0.067746,0.032162,0.041976,RES.COOKING,0.037377,0.047673,0.048136,0.023923,0.044188
3,RCHEHOB00 [RES.COOKING: .00.HOB.ELECTRIC.],UK,2010,164.642857142857,1,8.23214285714286,1,0,1,2.0,...,0.049751,0.067746,0.032162,0.041976,RES.COOKING,0.037377,0.047673,0.048136,0.023923,0.044188
4,RCHEHOB01 [RES.COOKING: .01.HOB.ELECTRIC.],UK,2010,164.642857142857,1,8.23214285714286,1,0,1,100000000.0,...,0.049751,0.067746,0.032162,0.041976,RES.COOKING,0.037377,0.047673,0.048136,0.023923,0.044188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,RWEAWHTRG00 [RES.WATER.EXISTING-AVERAGE: .00.N...,UK,2010,6.10855195224853,1,0.610855195224853,1,0,1,2.0,...,0.092418,0.0,0.052493,0.080124,NaN,0.052977,0.0,0.081446,0.0,0.0
132,RWEAWHTRG01 [RES.WATER.EXISTING-AVERAGE: .01.N...,UK,2010,6.10855195224853,1,0.610855195224853,1,0,1,100000000.0,...,0.092418,0.0,0.052493,0.080124,NaN,0.052977,0.0,0.081446,0.0,0.0
133,RWNASA01 [RES.WATER.NEW-AVERAGE: .01.STANDALON...,UK,2010,0.000593142810946307,1,0.000593142810946307,1,0,1,100000000.0,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.153112,0.0
134,RWNAWHTRE01 [RES.WATER.NEW-AVERAGE: .01.ELC.RE...,UK,2010,2.09714772485452,1,0.209714772485452,1,0,1,100000000.0,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.153112,0.0


In [41]:
final_merged_df.to_csv(data_folder / "Technodata_Ofgem.csv")